In [2]:
%pip install tensorflow
%pip install opencv-python
%pip install matplotlib

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  

In [5]:
import os

FALSE_PATH = os.path.join('images', 'false_images')

# Ungzipped Tar file in the Wild Dataset
#!tar -xf lfw.tgz

# Transfer images from lfw directory to images/false_images directory
path = 'lfw'
for folder in os.listdir(path):
    for img_file in os.listdir(os.path.join(path, folder)):
        try:
            CURRENT_PATH = os.path.join(path, folder, img_file)
            NEW_PATH = os.path.join(FALSE_PATH, img_file)
            os.replace(CURRENT_PATH, NEW_PATH)
        except FileNotFoundError as e:
            print(f"Error: {e}")
            print(f"File not found: {CURRENT_PATH}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")


In [13]:
import tensorflow as tf
import numpy as np
import os
import cv2
import uuid

In [12]:
TRUE_PATH=os.path.join('images','true_images')
FALSE_PATH=os.path.join('images','false_images')
INPUT_PATH=os.path.join('images','input_images')

In [22]:
def data_aug(img):
    data=[]
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6,upper= 1, seed=(1,3))
        img = tf.image.stateless_random_flip_left_right(img, seed = (np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90,max_jpeg_quality=100, seed = (np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9, upper=1,seed = (np.random.randint(100),np.random.randint(100)))

        data.append(img)

    return data

In [23]:
#input images augumentation
for file_name in os.listdir(os.path.join(INPUT_PATH)):
    img_path = os.path.join(INPUT_PATH, file_name)
    img = cv2.imread(img_path)
    aug_imgs = data_aug(img)

    for image in aug_imgs:
        cv2.imwrite(os.path.join(INPUT_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())
    
#true images augumentation
for file_name in os.listdir(os.path.join(TRUE_PATH)):
    img_path = os.path.join(TRUE_PATH, file_name)
    img = cv2.imread(img_path)
    aug_imgs = data_aug(img)

    for image in aug_imgs:
        cv2.imwrite(os.path.join(TRUE_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [24]:
input_fileset = tf.data.Dataset.list_files(INPUT_PATH+'/*.jpg').take(10)
true_fileset = tf.data.Dataset.list_files(TRUE_PATH+'/*.jpg').take(10)
false_fileset = tf.data.Dataset.list_files(FALSE_PATH+'/*.jpg').take(10)

In [25]:
input_fileset

<_TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [26]:
false_fileset

<_TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [27]:
true_fileset

<_TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [29]:
trueset = tf.data.Dataset.zip(input_fileset,true_fileset, tf.data.Dataset.from_tensor_slices(tf.ones(len(input_fileset))))
falseset = tf.data.Dataset.zip(input_fileset, false_fileset, tf.data.Dataset.from_tensor_slices(tf.ones(len(input_fileset))))
dataset = trueset.concatenate(falseset)
                              

In [35]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import Layer

In [67]:
def build_initial_model_layers(): 
    inp = Input(shape=(105,105,3), name='input_image')
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    model = Model(inputs=[inp], outputs=[d1], name='embedding')
    return model

In [68]:
class DistanceLayer(Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)
    
    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [71]:
def make_siamese_model(): 
    input_image = Input(name='input_img', shape=(105,105,3))
    validation_image = Input(name='validation_img', shape=(105,105,3))
    embedding = build_initial_model_layers()
    
    # Compute embeddings
    input_embedding = embedding(input_image)
    validation_embedding = embedding(validation_image)
    
    # Combine siamese distance components using the DistanceLayer
    siamese_layer = DistanceLayer()
    siamese_layer._name = 'distance'
    distances = siamese_layer(input_embedding, validation_embedding)
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    # Define the Siamese network model
    model = Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')
    
    return model

In [72]:
model_history = make_siamese_model()

In [73]:
model_history.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_img           │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ validation_img      │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4096)      │ 38,960,448 │ input_img[0][0],  │
│ (Functional)        │                   │            │ validation_img[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ distance_layer_9    │ (None, 4096)      │          0 │ embedding[0][0],  │
│ (DistanceLayer)     │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 1)         │      4,097 │ distance_layer_9… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)